In [4]:
import pandas as pd
import os

In [28]:
reviews=pd.read_csv('../datasets/csv/G_ulta_beauty.csv')

In [53]:
original=pd.read_parquet('../datasets/parquets/metadatos Google/metadatos_prelimpio08.parquet')

In [77]:
ulta_beauty=original[original.name=='Ulta Beauty']

In [78]:
ulta_beauty.url.unique()

array(['https://www.google.com/maps/place//data=!4m2!3m1!1s0x8644d7378e955b4d:0x332ab1014d32203a?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c259f5cd286f9f:0xd7f304c78a07bf1d?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x88e8bc9c1cb8a9c9:0xc54af76cc3aa026e?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c22f83fd079ae7:0xee3529b535dd9bd7?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c2f590aa8b7d31:0x924a1bbe0afca374?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x880fc7af7a4e4e75:0x642cff114e6f4196?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89e7ca02be0dbcf9:0x6ed41a5b33f76942?authuser=-1&hl=en&gl=us',
       'https://www.google.com/maps/place//data=!4m2!3m1!1s0x80dd2b9281580c83:0x35d93e9b1861b999?authuser=-1&hl=en&gl=us',
       'https://

In [79]:
ulta_beauty[ulta_beauty.url=='https://www.google.com/maps/place//data=!4m2!3m1!1s0x89e7ca02be0dbcf9:0x6ed41a5b33f76942?authuser=-1&hl=en&gl=us']

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,State
98732,Ulta Beauty,"Ulta Beauty, 470 Lewis Ave Ste 61, Meriden, CT...",0x89e7ca02be0dbcf9:0x6ed41a5b33f76942,Chain with a diverse selection of cosmetics & ...,41.551161,-72.807604,"[Cosmetics store, Beauty salon, Hair salon]",4.0,114,None,"[[Monday, 11AM–7PM], [Tuesday, 11AM–7PM], [Wed...",{'Accessibility': ['Wheelchair accessible entr...,Permanently closed,"[0x89e7ad9bfb587e17:0x94814216eb1d2745, 0x89e7...",https://www.google.com/maps/place//data=!4m2!3...,CT


In [80]:
len(ulta_beauty[ulta_beauty.state=='Permanently closed'])

2

In [81]:
len(ulta_beauty)-len(ulta_beauty[ulta_beauty.state=='Permanently closed'])

14

In [5]:
# Especifica la ruta de la carpeta que contiene los archivos .parquet particionados
ruta_carpeta_parquet = '../datasets/parquets/metadatos Google'

# Lista para almacenar DataFrames de cada partición
dataframes = []

# Itera sobre los archivos .parquet en la carpeta
for root, dirs, files in os.walk(ruta_carpeta_parquet):
    for file in files:
        if file.endswith('.parquet'):
            # Construye la ruta completa del archivo
            ruta_completa = os.path.join(root, file)
            
            # Lee el archivo .parquet y agrega el DataFrame a la lista
            df_particion = pd.read_parquet(ruta_completa)
            dataframes.append(df_particion)

# Concatena todos los DataFrames en uno solo
df_google = pd.concat(dataframes, ignore_index=True)

In [24]:
# Filtrar por el valor 'ulta beauty' en la columna 'name'
df_google = df_google[df_google['name'] == 'Ulta Beauty']
# Eliminamos columnas innecesarias
df_google = df_google[["gmap_id","latitude","longitude","avg_rating","State", "state", "address"]]

In [25]:
df_google.to_csv('../datasets/csv/G_beauty_establishments.csv', index=False)